In [1]:
from pathlib import Path
import zarr
from ome_zarr.io import parse_url
from ome_zarr.reader import Reader
from ome_zarr.writer import write_multiscale 
import dask.array as da
import importlib.metadata

In [2]:
# --- INPUTS ---
zarr_array_root = Path(r"D:/osw2025/slides-big-imaging-data-osw25/tutorials/pyramidal_chunked_human-neuron/level0")
voxel_size_l0   = [0.294, 0.241, 0.241]  # (z,y,x) micrometers
downsample_xy   = 2        # 2× downsample in Y & X per level
stop_min_xy     = 512      # stop when min(Y,X) <= this
is_labels       = False    # True for segmentation labels (uses max)
target_chunks   = None     # e.g. (1, 256, 256) for snappy slice viewing

# --- Load your plain Zarr ARRAY as dask ---
za   = zarr.open_array(str(zarr_array_root), mode="r")
arr0 = da.from_zarr(za)  # shape (Z, Y, X)
if target_chunks:
    arr0 = arr0.rechunk(target_chunks)

# --- Downsample helper (coarsen over Y,X; Z unchanged) ---
def ds_xy(a, f=2, agg="mean"):
    fac = {0: 1, 1: f, 2: f}
    if agg == "mean":
        out = da.coarsen(da.mean, a, fac, trim_excess=True).astype(a.dtype)
    elif agg == "max":
        out = da.coarsen(da.max, a, fac, trim_excess=True)
    else:
        raise ValueError("agg must be 'mean' (intensity) or 'max' (labels)")
    return out

agg = "max" if is_labels else "mean"

# --- Build pyramid levels: L0..Ln ---
levels = [arr0]
while True:
    _, y, x = levels[-1].shape
    if min(y, x) <= stop_min_xy:
        break
    nxt = ds_xy(levels[-1], f=downsample_xy, agg=agg)
    if target_chunks:
        nxt = nxt.rechunk(target_chunks)
    levels.append(nxt)

print(f"Built {len(levels)} level(s): L0 shape={tuple(levels[0].shape)} → L{len(levels)-1} shape={tuple(levels[-1].shape)}")

# --- NGFF metadata (axes need 'type') ---
axes_meta = [
    {"name": "z", "type": "space", "unit": "micrometer"},
    {"name": "y", "type": "space", "unit": "micrometer"},
    {"name": "x", "type": "space", "unit": "micrometer"},
]
coord_tx = []
for i in range(len(levels)):
    scl = [
        voxel_size_l0[0],                  # Z unchanged
        voxel_size_l0[1] * (downsample_xy ** i),
        voxel_size_l0[2] * (downsample_xy ** i),
    ]
    coord_tx.append([{"type": "scale", "scale": scl}])

# --- Output NGFF store ---
out_path = zarr_array_root.parent / (zarr_array_root.name + ".pyramid.ngff.zarr")
out_grp  = zarr.open_group(str(out_path), mode="w")

# --- Write pyramid multiscale ---
try:
    write_multiscale(
        pyramid=levels,
        group=out_grp,
        axes=axes_meta,
        coordinate_transformations=coord_tx,
        compute=True,                       # materialize data to disk
    )
except TypeError:
    # Older ome-zarr signature (positional first arg)
    write_multiscale(
        levels,
        out_grp,
        axes=axes_meta,
        coordinate_transformations=coord_tx,
        compute=True,
    )

# Optional: consolidate for faster reading
zarr.consolidate_metadata(str(out_path))

print("ome-zarr version:", importlib.metadata.version("ome-zarr"))
print(f"✅ Wrote pyramidal NGFF store to: {out_path}")

Built 4 level(s): L0 shape=(527, 2614, 2344) → L3 shape=(527, 326, 293)
ome-zarr version: 0.11.1
✅ Wrote pyramidal NGFF store to: D:\osw2025\slides-big-imaging-data-osw25\tutorials\pyramidal_chunked_human-neuron\level0.pyramid.ngff.zarr


In [3]:
# --- 1. Path to your local directory ---
zarr_path = Path("D:/osw2025/slides-big-imaging-data-osw25/tutorials/pyramidal_chunked_human-neuron/level0.pyramid.ngff.zarr").resolve()
print(f"Opening OME-Zarr from: {zarr_path}")

Opening OME-Zarr from: D:\osw2025\slides-big-imaging-data-osw25\tutorials\pyramidal_chunked_human-neuron\level0.pyramid.ngff.zarr


In [4]:
# --- 2. Open with ome-zarr-py ---
url = parse_url(str(zarr_path), mode="r")
reader = Reader(url)
nodes = list(reader())

print(f"Found {len(nodes)} node(s) in this store")

Found 1 node(s) in this store


In [5]:
# --- 3. Optional: view Zarr group structure ---
import zarr
import sys
try:
    root = zarr.open_group(str(zarr_path), mode="r")
    print("\nGroup tree:")
    print(root.tree())
except Exception as e:
    print(f"Error opening Zarr group or printing tree: {e}")


Group tree:
/
 ├── 0 (527, 2614, 2344) uint8
 ├── 1 (527, 1307, 1172) uint8
 ├── 2 (527, 653, 586) uint8
 └── 3 (527, 326, 293) uint8


In [7]:
import json
root = zarr.open_group(str(zarr_path), mode="r")
print(root.tree())
print(json.dumps(json.loads((zarr_path/".zattrs").read_text()), indent=2))

/
 ├── 0 (527, 2614, 2344) uint8
 ├── 1 (527, 1307, 1172) uint8
 ├── 2 (527, 653, 586) uint8
 └── 3 (527, 326, 293) uint8
{
  "multiscales": [
    {
      "axes": [
        {
          "name": "z",
          "type": "space",
          "unit": "micrometer"
        },
        {
          "name": "y",
          "type": "space",
          "unit": "micrometer"
        },
        {
          "name": "x",
          "type": "space",
          "unit": "micrometer"
        }
      ],
      "datasets": [
        {
          "coordinateTransformations": [
            {
              "scale": [
                0.294,
                0.241,
                0.241
              ],
              "type": "scale"
            }
          ],
          "path": "0"
        },
        {
          "coordinateTransformations": [
            {
              "scale": [
                0.294,
                0.482,
                0.482
              ],
              "type": "scale"
            }
          ],
    

In [8]:
import json
from pathlib import Path

#zarr_path = Path("your_store_path")
zattrs_file = zarr_path / ".zattrs"
if zattrs_file.exists():
    zattrs = json.loads(zattrs_file.read_text())
    is_pyramidal = "multiscales" in zattrs and len(zattrs["multiscales"][0].get("datasets", [])) > 1
    print("Is pyramidal:", is_pyramidal)
else:
    print(".zattrs file not found")

Is pyramidal: True


In [9]:
import zarr, dask.array as da, napari

root = zarr.open_group(str(zarr_path), mode="r")
arr0 = da.from_zarr(root["0"])  # this is your (527, 2614, 2344) array

viewer = napari.Viewer()
viewer.add_image(arr0, name="L0", scale=[0.294, 0.241, 0.241])  # (z,y,x) voxel size µm
napari.run()
